In [4]:
import MLCE_CWBO2025.virtual_lab as virtual_lab
import numpy as np
import scipy
import random
import sobol_seq
import time
from datetime import datetime

In [5]:
group_names     = ['Idrees Kholwadia', 'Joe Wood', 'Joseph Wright', 'Kundan Mahitkumar']
cid_numbers     = ['000000','111111', '22222', '02219861']
oral_assessment = [0, 1]

In [6]:
def objective_func(X):
    return (np.array(virtual_lab.conduct_experiment(X)))

category_map = {
    'celltype_1': [1, 0, 0],
    'celltype_2': [0, 1, 0],
    'celltype_3': [0, 0, 1]
    }

In [7]:
X_initial = ([[31.250000,  7.500000,  37.500000,   4.166667,  20.833333, 'celltype_3'],
              [36.250000,  6.500000,   4.166667,  37.500000,  45.833333,  'celltype_2'],
              [33.750000, 7.166667,  20.833333,  12.500000,  12.500000, 'celltype_1'],
              [38.750000,  6.166667,  12.500000,  29.166667,  29.166667, 'celltype_3'],
              [37.500000,  7.833333,  45.833333,  20.833333,   4.166667, 'celltype_1'],
              [34.166667,  6.833333,  29.166667,  45.833333,  37.500000, 'celltype_2'],
              [31.500000,  6.833333,  0.833333,  0.833333,   42.166667, 'celltype_3'],
              [32.500000,  6.133333,  4.833333,  40.833333,   30.166667, 'celltype_2'],
              [34.500000,  7.933333,  5.833333,  38.833333,   23.166667, 'celltype_1'],
              [35.500000,  6.233333,  9.833333,  49.833333,   8.166667, 'celltype_3'],
              [39.500000,  7.433333,  45.833333,  23.833333,   27.166667, 'celltype_2'],
              [37.500000,  7.133333,  35.833333,  12.833333,   42.166667, 'celltype_1'],])

Y_initial = objective_func(X_initial)
print(Y_initial)

Y_initial = np.array(Y_initial)

X_continuous = np.array([row[:-1] for row in X_initial])
X_categorical = [row[-1] for row in X_initial]
X_categorical_value = np.array([category_map[i] for i in X_categorical])

X_initial = np.hstack([X_continuous, X_categorical_value])
Y_initial = Y_initial.reshape(-1,1)
Y_initial

[ 1.29379748 47.92251709 18.08443557 14.96160162  1.29399739 88.5884237
 26.32980004 66.75440325  1.31316795 41.57018035  1.30573718  1.4756528 ]


array([[ 1.29379748],
       [47.92251709],
       [18.08443557],
       [14.96160162],
       [ 1.29399739],
       [88.5884237 ],
       [26.32980004],
       [66.75440325],
       [ 1.31316795],
       [41.57018035],
       [ 1.30573718],
       [ 1.4756528 ]])

In [8]:
temp = np.linspace(30, 40, 5)
pH   = np.linspace(6, 8, 5)
f1   = np.linspace(0, 50, 5)
f2   = np.linspace(0, 50, 5)
f3   = np.linspace(0, 50, 5)
celltype = ['celltype_1','celltype_2','celltype_3']

X_searchspace = [[a,b,c,d,e,f] for a in temp for b in pH for c in f1 for d in f2 for e in f3 for f in celltype]

In [ ]:
class GP:
    def __init__(self, X, Y, kernel, multi_hyper, var_out=True): 
        self.X, self.Y, self.kernel = X, Y, kernel
        self.n_point, self.nx_dim = X.shape[0], X.shape[1]
        self.ny_dim = Y.shape[1]
        self.multi_hyper = multi_hyper
        self.var_out = var_out

        self.X_mean, self.X_std = np.mean(X, axis=0), np.std(X, axis=0) + 1e-8
        self.Y_mean, self.Y_std = np.mean(Y, axis=0), np.std(Y, axis=0) + 1e-8
        self.X_norm, self.Y_norm = (X - self.X_mean)/self.X_std, (Y - self.Y_mean)/self.Y_std

        self.hypopt, self.invKopt = self.determine_hyperparameters()

    # covariance matrix
    def cov_mat(self, kernel, X_norm, W, sf2):
        if kernel == 'RBF':
            dist = scipy.spatial.distance.cdist(X_norm, X_norm, 'seuclidean', V=W)**2
            cov_matrix = sf2*np.exp(-0.5*dist)
            return cov_matrix

    # single set xnorm against the dataset Xnorm
    def cal_cov_matrix(self, xnorm, Xnorm, ell, sf2):
        nx_dim = self.nx_dim

        dist = scipy.spatial.distance.cdist(xnorm.reshape(1, nx_dim), Xnorm, 'seuclidean', V=ell)**2
        cov_matrix = sf2*np.exp(-.5 * dist)

        return cov_matrix
    
    def negative_loglikelihood(self, hyper, X, Y):
        n_point, nx_dim = self.n_point, self.nx_dim
        kernel = self.kernel

        W = np.exp(2*hyper[:nx_dim])
        sf2 = np.exp(2*hyper[nx_dim])
        sn2 = np.exp(2*hyper[nx_dim+1])

        K = self.cov_mat(kernel, X, W, sf2)
        K = K + (sn2 + 1e-8)*np.eye(n_point)
        K = (K + K.T)*0.5
        L = np.linalg.cholesky(K)
        logdetK = 2 * np.sum(np.log(np.diag(L)))
        invLY = np.linalg.solve(L, Y)
        alpha = np.linalg.solve(L.T, invLY)
        NLL = np.dot(Y.T, alpha) + logdetK
        
        return NLL
    
    def determine_hyperparameters(self):
        multi_start = 5
        lb = np.array([1e-1, 1e-1, 1e-1, 1e-1, 1e-1, 1e-1, 1e-1, 1e-1, 1e-3, 1e-6])
        ub = np.array([3, 3, 3, 3, 3, 3, 3, 3, 10, 0.010])
        num_hyperparameters = 10
        multi_startvec = sobol_seq.i4_sobol_generate(num_hyperparameters, multi_start)
        localsol = np.empty((multi_start, num_hyperparameters))
        localval = np.empty(multi_start)        
        hypopt = np.zeros((10, 1))
        options = {
            'disp': True,
            'maxiter': 1000,
        }
        invKopt = []
        for i in range(self.ny_dim):
            for j in range(multi_start):
                print('multistart hyper parameter optimisation iteration= ', j,' input= ', i)
                hyp_init = lb + (ub-lb)*multi_startvec[j,:]
                res = scipy.optimize.minimize(self.negative_loglikelihood, hyp_init, args=(self.X_norm, self.Y_norm[:,i]), method='SLSQP', 
                                              options=options, bounds=scipy.optimize.Bounds(lb, ub), tol=1e-12)
                localsol[j] = res.x
                localval[j] = res.fun

            minindex = np.argmin(localval)
            hypopt[:,i] = localsol[minindex]
            ellopt = np.exp(2.*hypopt[:self.nx_dim,i])
            sf2opt = np.exp(2.*hypopt[self.nx_dim, i])
            sn2opt = np.exp(2.*hypopt[self.nx_dim+1, i]) + 1e-8

            Kopt = self.cov_mat(self.kernel, self.X_norm, ellopt, sf2opt) + sn2opt*np.eye(self.n_point)
            Kopt = (Kopt + Kopt.T) * 0.5 # Symmetrize
            invKopt += [np.linalg.solve(Kopt, np.eye(self.n_point))]

        return hypopt, invKopt
    
    def GP_inference_np(self, x):
        xnorm = (x - self.X_mean)/self.X_std
        mean = np.zeros(self.ny_dim)
        var = np.zeros(self.ny_dim)

        for i in range(self.ny_dim):
            invK = self.invKopt[i]
            hyper = self.hypopt[:,i]
            ellopt, sf2opt = np.exp(2*hyper[:self.nx_dim]), np.exp(2*hyper[self.nx_dim])

            k = self.cal_cov_matrix(xnorm, self.X_norm, ellopt, sf2opt)
            mean[i] = np.matmul(np.matmul(k, invK), self.Y_norm[:,i]).item()
            var[i] = max(0, sf2opt - (np.matmul(np.matmul(k, invK), k.T).item()))
        
        mean_sample = mean*self.Y_std + self.Y_mean
        var_sample = var*self.Y_std**2

        if self.var_out:
            return mean_sample, var_sample
        else:
            return mean_sample.flatten()[0]

In [ ]:
gp_model = GP(X_initial, Y_initial, 'RBF', 5) 

print(gp_model.hypopt)

mean1, var1 = gp_model.GP_inference_np(np.array([[3, 6, 10, 20, 20, 0, 0, 1]]))

print(mean1, var1)


multistart hyper parameter optimisation iteration=  0  input=  0
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.913594837789638
            Iterations: 25
            Function evaluations: 232
            Gradient evaluations: 21
multistart hyper parameter optimisation iteration=  1  input=  0
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.913594837788995
            Iterations: 43
            Function evaluations: 485
            Gradient evaluations: 43
multistart hyper parameter optimisation iteration=  2  input=  0
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.913594837788828
            Iterations: 27
            Function evaluations: 254
            Gradient evaluations: 23
multistart hyper parameter optimisation iteration=  3  input=  0
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.91359483778928

In [ ]:
class BO:
    def __init__(self, X_initial, X_searchspace, iterations, batch, objective_func):
    
        start_time = datetime.timestamp(datetime.now())

        self.Y    = []  # output data appended after evaluating input conditions with the objective function 
        self.time = []  # runtime data appended after each loop

        self.X_initial     = X_initial
        self.X_searchspace = X_searchspace
        self.iterations    = iterations
        self.batch         = batch

        self.Y    += [evaluations]
        self.time += [datetime.timestamp(datetime.now())-start_time] # remember to append the appropriate number of 0 ms according to batch size
        start_time = datetime.timestamp(datetime.now())

In [ ]:
BO_m = BO(X_initial, X_searchspace, 15, 5, objective_func)